# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
import pickle

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM Clean_ELT_pipeline', con = engine)
df.head()

,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
y = df.drop(['id','message','original', 'genre'], axis = 1)
y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # detect all URL present in the messages
    detected_urls = re.findall(url_regex, text)
    # replace URL with "urlplaceholder"
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('cvect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)
y_pred_df.head()

,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)","('search_and_rescue',)","('security',)","('military',)","('child_alone',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y_pred_df.shape

(6554, 36)

In [11]:

for column in y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(y_test[column], y_pred_df[column]))

column_name: ('related',)

             precision    recall  f1-score   support

          0       0.59      0.35      0.43      1481
          1       0.82      0.93      0.87      5029
          2       0.67      0.05      0.09        44

avg / total       0.77      0.79      0.77      6554

column_name: ('request',)

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5412
          1       0.83      0.36      0.50      1142

avg / total       0.87      0.88      0.85      6554

column_name: ('offer',)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

column_name: ('aid_related',)

             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3828
          1       0.74      0.53      0.62      2726

avg / total       0.73      0.7

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f1068841378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None

In [18]:
parameters = {
    'clf__estimator__n_estimators': [ 10, 15],
    'clf__estimator__min_samples_split': [2, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2,)

In [19]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  38.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.5s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  38.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  39.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, total=  53.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, total=  53.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15, total=  54.2s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, total=  35.8s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV]  clf_

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 10.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 15], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [20]:
cv.cv_results_

{'mean_fit_time': array([ 34.54824559,  49.2133526 ,  31.02124516,  43.84806927]),
 'std_fit_time': array([ 0.31284187,  0.35582302,  0.59849664,  0.79623887]),
 'mean_score_time': array([ 3.97069709,  4.5444986 ,  4.08542911,  4.57010245]),
 'std_score_time': array([ 0.07593882,  0.12519143,  0.04246855,  0.10846983]),
 'param_clf__estimator__min_samples_split': masked_array(data = [2 2 4 4],
              mask = [False False False False],
        fill_value = ?),
 'param_clf__estimator__n_estimators': masked_array(data = [10 15 10 15],
              mask = [False False False False],
        fill_value = ?),
 'params': [{'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 10},
  {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 15},
  {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 10},
  {'clf__estimator__min_samples_split': 4,
   'clf__estimator__n_estimators': 15}],
 'split0_test_score': array([ 0.21452548,  0.2

In [21]:
#finding the best paramesters based on grip search
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 15}


In [22]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = optimised_model.predict(X_test)

for column in y_test.columns:
    print('column_name: {}\n'.format(column))
    print(classification_report(y_test[column], y_pred_df[column]))

column_name: ('related',)

             precision    recall  f1-score   support

          0       0.59      0.35      0.43      1481
          1       0.82      0.93      0.87      5029
          2       0.67      0.05      0.09        44

avg / total       0.77      0.79      0.77      6554

column_name: ('request',)

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5412
          1       0.83      0.36      0.50      1142

avg / total       0.87      0.88      0.85      6554

column_name: ('offer',)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

column_name: ('aid_related',)

             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3828
          1       0.74      0.53      0.62      2726

avg / total       0.73      0.7

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [29]:
# Try using SVM instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('cvect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])



In [30]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f1068841378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None

In [31]:
parameters2 = {'clf__estimator__n_estimators': [ 10, 15],
    'clf__estimator__min_samples_split': [2, 4]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, verbose = 10)

# Find best parameters
np.random.seed(81)
tuned_model2 = cv2.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.2142203234665853, total=  39.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   47.9s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.22993591699725358, total=  38.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.21940799511748552, total=  37.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=15 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  7.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, score=0.21269453768690877, total=  34.9s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  7.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15, score=0.21498321635642356, total=  47.8s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15, score=0.2216966737870003, total=  46.8s
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=15, score=0.21177906621910284, total=  48.8s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 10.6min finished


### 9. Export your model as a pickle file

In [24]:
pickle.dump(optimised_model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.